# Converting from decimal to binary and back

## 1 Representations of number in different bases

It is important to differentiate a real number $ x $ from the form of its representation (binary, decimal, octal, hexadecimal, etc.). Any *integer* $ b \ge 2 $ can serve as the **base** of a representation system. In this system, numbers are represented using the notation

$$ x = (a_m\, a_{m - 1}\dots a_1\, a_0\,.\,a_{-1}\, a_{-2} \dots)_{b}\ , $$

which means that

$$ x = a_mb^m + a_{m - 1}b^{m - 1} + \dots + a_1 b + a_0 
   + a_{-1}b^{-1} + a_{-2}b^{-2} + \dots. $$
   
The number of digits to the *left* of the point is always *finite*, because any real number is bounded. In contrast, the number of digits to the *right* of the point can be *infinite*, because not every number is rational.

📝 Because we commonly use the decimal system almost exclusively to represent numbers, we write, say, $ x = 592 $ instead of $ x = (592)_{10} $. However, it is important to keep in mind that '$ 592 $' is not the number $ x $ itself, but rather its *representation* in in base $ 10 $. The same number could also be written as $ x = (1001010000)_2 $, and were the binary system the one most commonly used, we would instead abuse our notation to write $ x = 1001010000 $.

**Exercise:** Let $ b > 1 $ be an integer. Prove that the representation of a real number $ x $ is finite in base $ b $ if and only if $ x $ can be expressed as a fraction in which the denominator is of the form $ b^r $ for some integer $ r \ge 0 $. 

Most computing uses the **binary** form of representing numbers, in which the base $ b $ equals $ 2 $. There are several reasons for this, one being that this is the simplest possible choice; another is the close relationship between Boolean logic and the arithmetic of integers modulo 2.

In the binary system, the digit of a number is usually referred to as a **bit** (short for _binary digit_). Our immediate objective is to understand how to convert from the binary system to the more familiar decimal system and back.

For example, 

\begin{align} (1001.101)_2 &= 1 \times 2^3 + 0 \times 2^2 + 0 \times 2^1 + 1 \times 2^0 + 1 \times 2^{-1} + 0 \times 2^{-2} + 1 \times 2^{-3}  \\
& = 8 + 1 + \frac{1}{2} + \frac{1}{8} \\ &= \frac{77}{8}.
\end{align}

## 2 Computing a number from its representation in base $ b $<a name="section2"></a>

### 2.1 Obtaining an integer from its representation in base $ b $
**Problem:** Define a function which takes a base $ b $ and the *representation* of a positive integer $ n $ in this base, given as a string of digits between $ 0 $ and $ b - 1 $, and which returns $ n $ as output.

In [1]:
def rep_to_number(b: int, digits: str) -> int:   # The type annotations are optional!
    """A function which takes a string of digits between 0 and b - 1, where the base b >= 1
    is an integer, and returns the integer whose representation in base b is the given one."""
    
    assert isinstance(b, int) and b >= 2         # Make sure b is an integer >= 2.
    allowed = [str(d) for d in range(0, b)]      # Allowed characters for representations in base b.
    for digit in digits:                         # Check if the representation is valid.
        if digit not in allowed:
            raise ValueError(f"Invalid representation of a number in base {b}.")
       
    highest_exponent = len(digits) - 1
    n = 0
    exponent = highest_exponent
    for digit in digits:
        n += int(digit) * b**exponent    # Add to n the current digit multiplied by the current exp.
        exponent -= 1
    
    return n


print(rep_to_number(2, "1011"))
print(rep_to_number(3, "1011"))
print(rep_to_number(10, "1011"))

11
31
1011


### 2.2 Obtaining a fractional number from its representation in base $ b $
**Problem:** Define a function which takes a base $ b $ and the (finite) *representation* of a number $ t $ ($ 0 \le t < 1 $) in this base, given as a string of digits between $ 0 $ and $ b - 1 $, and returns $ t $ as output.

*Solution:* We can easily solve this by making use of the function defined in the preceding section as follows. Since $ 0 \le t < 1 $ by hypothesis, its representation in base $ b $ must be of the form
$$ t = (.a_{-1}\,a_{-2}\dots a_{-m})_b $$
for some $ m \ge 1 $. Equivalently,
$$ t = a_{-1}b^{-1} + a_{-2}b^{-2} + \dots + a_{-m}b^{-m} .$$
Multiplying both sides by $ b^ m $:
$$ b^m t = a_{-1}b^{m - 1} + a_{-2}b^{m - 2} + \dots + a_{-m} b^0 .$$
In other words, the *integer* $b^m t $ has the representation
$$ b^m t = (a_{-1}\,a_{-2}\dots a_{-m})_b .$$
(Note the absence of the point in the latter representation.) Therefore, we can obtain the value of $ t $ by first computing the *integer* $ b^m t $ using the preceding representation and then multiplying the result by $ b^{-m} $.

In [2]:
def rep_to_fractional(b: int, digits: str) -> int:   # The type annotations are optional!
    """A function which takes a string of digits between 0 and b - 1,
    where the base b >= 1 is an integer, and returns the fractional number
    whose representation in base b is the given one."""
    m = len(digits)
    n = rep_to_number(b, digits)
    return n * b**(-m)


print(rep_to_fractional(2, "11"))
print(rep_to_fractional(3, "11"))
print(rep_to_fractional(10, "11"))

0.75
0.4444444444444444
0.11


### 2.3 Obtaining a real number from its representation in base $ b $

**Exercise:** Define a function which takes three arguments:
* The base $ b $;
* The representation of the integral part $ n \ge 1 $ of a real number $ x $ in base $ b $;
* The representation of the fractional part $ 0 \le t < 1 $ of a real number $ x $ in base $ b $;

and which returns $ x $ as output. (*Hint:* Use the functions that were defined above.)

**Exercise:** How should each of the functions above be modified to include the possibility that $ x \le 0 $?

## 3 Computing the representation of a number in base $ b $

We will now consider the converse problem to the one in [$ \S 2 $](#section2), namely, how to compute the representation of a number in base $ b $ given the number $ x $ itself.

### 3.1 Obtaining a the representation in base $ b $ of an integer

**Problem:** Define a function which takes a base $ b $ and a positive integer $ n \ge 1 $ and returns the representation of $ n $ in this base, given as a string of digits between $ 0 $ and $ b - 1 $.

*Solution*: Consider the task of finding the binary representation of the number $ n = 63 $. A smart strategy is to assume that we already have this representation:
$$ n = (a_m\,a_{m-1}\dots a_1a_0)_2 $$
and to consider how each of these digits (bits, in this case) can be recovered from $ n $. We are given, in other words, that
$$ n = a_m\times 2^m + a_{m-1} \times 2^{m-1} + \dots + a_1 \times 2^1 + a_0\times 2^0 .$$
Now note that:
* $ a_0 \equiv n \pmod 2 $, that is, $ a_0 $ is the remainder of $ n $ upon division by $ 2 $.

### 3.1 Obtaining a the representation in base $ b $ of a fractional number

**Problem:** Let $ 0 \le t < 1 $ be a fractional number and $ b $ be an integer $ \ge 2 $. Write a function to find the representation of $ t $ in base $ b $, given as a string of digits between $ 0 $ and $ b - 1 $.

*Solution*: Let 
$$ t = (.a_{-1}\,a_{-2}\dots a_{-m}\dots)_b $$
be this representation, that is,
$$ t = a_{-1}b^{-1} + a_{-2}b^{-2} + \dots + a_{-m}b^{-m} + \dots .$$
Multiplying both sides by $ b $, we deduce that
$$ bt = a_{-1} + \big( a_{-2}b^{-1} + \dots + a_{-m}b^{-m + 1} + \dots \big) .$$
Hence the *first digit $ a_{-1} $ in this representation is the integral part of $ bt $*, while the terms in parentheses form the fractional part of $ bt $. We can multiply the latter by $ b $ and take the integral part of the result to obtain $ a_{-2} $. Proceeding in the same way, we can compute as many of the digits $ a_k $ as we like. The corresponding procedure is implemented below.

In [9]:
def number_to_rep(b: int, n: int) -> str:
    """Given a base b and a nonnegative integer n, returns the
    representation of n in base b as a string of digits from 0 to (b - 1)."""
    assert isinstance(b, int) and b >= 2
    assert isinstance(n, int) and n >= 0
    
    list_of_digits = []                   # Will store the list of digits of n.
    while n > 0: 
        d = n % b
        n //= b                           # The new value of n is the quotient of n by b.
        list_of_digits.append(str(d))     # Convert d to string, append it.
    # After the computation has finished, convert the list into a string by
    # joining its elements to the empty string.
    list_of_digits.reverse()              # Reverse the list of digits.
    representation = ''.join(list_of_digits)
    return representation


print(number_to_rep(2, 63))
print(number_to_rep(2, 64))

111111
1000000


In [6]:
def fractional_to_rep(b: int, t: float, max_iter: int) -> str:
    """Given a base b and a nonnegative integer n, returns the
    representation of n in base b as a string of digits from 0 to (b - 1)."""
    assert isinstance(b, int) and b >= 2
    assert isinstance(t, float) and 0 <= t < 1
    
    iterations = 0
    list_of_digits = []                   # Will store the list of digits of t.
    while t > 0 and iterations < max_iter:
        iterations += 1
        t *= b
        d = int(t)
        t %= 1
        list_of_digits.append(str(d))     # Convert d to string, append it.
    
    # After the computation has finished, convert the list into a string by
    # joining its elements to the empty string.
    representation = ''.join(list_of_digits)
    return representation


print(fractional_to_rep(2, 1 / 8, 10))
print(fractional_to_rep(10, 1 / 3, 10))


001
3333333333


We saw in the preceding section how to compute a real number given its representation in any base $ b \ge 2 $. We would now like to consider the problem of converting from the binary to the decimal representation (and back) directly.

**Problem:** Let $ (a_m\,a_{m-1}\dots a_1\,a_0)_2 $ be the 

## 3 Converting a float from decimal to binary

## 4 Converting a float from binary to decimal